In [1]:
%defaultDatasource jdbc:h2:mem:db

In [2]:
DROP TABLE IF EXISTS Drug;
DROP TABLE IF EXISTS Pathology;
DROP TABLE IF EXISTS DrugUse;

CREATE TABLE Drug (
  code INT NOT NULL,
  name VARCHAR(200) NOT NULL,
  PRIMARY KEY(code)
) AS SELECT
    code,
    name
FROM CSVREAD('../../../data/faers-2017/drug.csv');

CREATE TABLE Pathology (
  code INT NOT NULL,
  name VARCHAR(200) NOT NULL,
  PRIMARY KEY(code)
) AS SELECT
    code,
    name
FROM CSVREAD('../../../data/faers-2017/pathology.csv');

CREATE TABLE DrugUse (
  idPerson INT NOT NULL,
  codePathology INT NOT NULL,
  codeDrug INT NOT NULL,
  FOREIGN KEY(codePathology)
    REFERENCES Pathology(code)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION,
  FOREIGN KEY(codeDrug)
    REFERENCES Drug(code)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION,
) AS SELECT
    idPerson,
    codePathology,
    codeDrug
FROM CSVREAD('../../../data/faers-2017/drug-use.csv');

CREATE TABLE SideEffect(
  idPerson INT NOT NULL,
  codePathology INT NOT NULL,
  FOREIGN KEY(codePathology)
    REFERENCES Pathology(code)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION
) AS SELECT idPerson, codePathology
FROM CSVREAD('../../../data/faers-2017/sideeffect.csv');

In [3]:
SELECT * FROM Drug;
SELECT * FROM Pathology;
SELECT * FROM DrugUse;
SELECT * FROM SideEffect;

# FDA Adverse Event Reporting System (FAERS)

[FDA Adverse Event Reporting System (FAERS)](https://open.fda.gov/data/faers/).

Fonte de dados: https://www.fda.gov/drugs/fda-adverse-event-reporting-system-faers/fda-adverse-event-reporting-system-faers-latest-quarterly-data-files

# Interação entre medicamentos
Considere como uma interação toda vez que um medicamento A é tomado em conjunto com um medicamento B por uma pessoa. Crie uma consulta que retorne para cada par de medicamento A,B o número de interações, somente para aqueles medicamentos que são tomados em conjunto mais de 30 vezes (passaremos a chamar de **grupo mais 30**).

In [ ]:
DROP VIEW IF EXISTS Interaction;
DROP VIEW IF EXISTS Interaction30;

CREATE VIEW Interaction AS
SELECT DUA.codeDrug AS drugA, DUB.codeDrug AS drugB
FROM DrugUse AS A, DrugUse AS B
WHERE A.IdPerson = B.IdPerson AND A.codeDrug < B.codeDrug;

CREATE VIEW Interaction30 AS
SELECT drugA, drugB, count(*) weight
FROM Interaction
GROUP BY drugA, drugB
HAVING weight > 30;

SELECT * FROM Interaction30;

## Tabela Resultante

Se estiver demorando muito para gerar a tabela, você poderá usar a seguinte tabela resultante pronta. Essa tabela tem todas as interações, você ainda precisará filtrar o **grupo mais 30**.

In [5]:
CREATE TABLE DrugInteraction (
  druga INT NOT NULL,
  drugb INT NOT NULL,
  weight INT NOT NULL,
  PRIMARY KEY(druga, drugb)
) AS SELECT
    druga,
    drugb,
    weight
FROM CSVREAD('../../../data/faers-2017/results/drug-interaction.csv');

org.h2.jdbc.JdbcSQLException:  Table "DRUGINTERACTION" already exists; SQL statement

In [6]:
DROP VIEW DrugInteraction30 IF EXISTS;
CREATE VIEW DrugInteraction30 AS
SELECT * FROM DrugInteraction
WHERE DrugInteraction.weight > 30
ORDER BY druga, drub;

SELECT * FROM DrugInteraction30;

org.h2.jdbc.JdbcSQLException:  Column "DI.WEIGHT" not found; SQL statement

# Suporte

### Qual o suporte que cada uma das associações tem?

Gere uma consulta que apresente o suporte do **grupo mais 30**. Ordene o resultado por ordem de suporte. 

Observação: Para que o resultado da divisão seja fracionária, converta o numerador em `DOUBLE` usando:
~~~sql
CAST(campo AS DOUBLE)
~~~

In [ ]:
SELECT COUNT(*) FROM DrugInteraction;

In [ ]:
SELECT druga, drugb, weight, CAST(weight AS DOUBLE) / 9073 Sup
FROM DrugInteraction30
ORDER BY Sup;

# Confiança

### Qual a confiança que cada uma das associações tem?

Gere uma consulta que apresente a confiança do **grupo mais 30**. Ordene o resultado por ordem de confiança.

In [ ]:
SELECT Interaction30.druga, CAST(count(*) AS DOUBLE)/(SELECT count(*) FROM Interaction) AS trust
FROM Interaction, Interaction30
WHERE Interaction.druga = Interaction30.druga
GROUP BY Interaction.druga
ORDER BY trust;

# Efeito Colateral

Para cada medicamento, qual o percentual do medicamento A que tem efeito colateral junto com o medicamento B, quando comparado com seu uso sem o medicamento B.

In [ ]:
CREATE VIEW ReducedDrugUse AS
SELECT *
FROM DrugUse
ORDER BY idPerson
LIMIT 300;

CREATE VIEW DrugsPerPerson AS
SELECT idPerson,count(codeDrug) as drugCount
FROM ReducedDrugUse
GROUP BY idPerson;

CREATE VIEW DrugAlone AS
SELECT ReducedDrugUse.idPerson, ReducedDrugUse.codeDrug
FROM ReducedDrugUse, DrugsPerPerson
WHERE ReducedDrugUse.idPerson = DrugsPerPerson.idPerson AND DrugsPerPerson.drugCount = 1;

CREATE VIEW DrugIntersection AS
SELECT Du1.idPerson, Du1.codeDrug as druga, Du2.codeDrug AS drugb
FROM ReducedDrugUse AS DU1, ReducedDrugUse AS DU2
WHERE DU1.idPerson = DU2.idPerson AND Du1.codeDrug <> Du2.codeDrug AND DU1.codeDrug < DU2.codeDrug


CREATE VIEW NumberOfSideEffectsByDrug AS
SELECT codeDrug,count(codeDrug) as sideeffects
FROM DrugAlone, SideEffect
WHERE DrugAlone.idPerson = SideEffect.idPerson
GROUP BY DrugAlone.codeDrug;

CREATE VIEW NumberOfUsesByDrug AS
SELECT codeDrug,count(DrugAlone.codeDrug) as uses
FROM DrugAlone
GROUP BY codeDrug;

CREATE VIEW AloneSideEffectFrequency AS
SELECT NumberOfUsesByDrug.codeDrug, CAST(NumberOfUsesByDrug.uses as DOUBLE)/ NumberOfSideEffectsByDrug.sideeffects as sideeffectfreq
FROM NumberOfSideEffectsByDrug,NumberOfUsesByDrug
WHERE NumberOfUsesByDrug.codeDrug = NumberOfSideEffectsByDrug.codeDrug


CREATE VIEW NumberOfSideEffectsByDrugCombination AS
SELECT druga,drugb,count(*) as sideeffects
FROM DrugIntersection, SideEffect
WHERE DrugIntersection.idPerson = SideEffect.idPerson
GROUP BY druga,drugb;


CREATE VIEW NumberOfUsesByDrugCombination AS
SELECT druga,drugb,count(*) as uses
FROM DrugIntersection
GROUP BY druga,drugb;

CREATE VIEW CombinedSideEffectFrequency AS
SELECT SideComb.druga, SideComb.drugb, CAST(NUses.uses AS DOUBLE)/SideComb.sideeffects as sideeffectfreq
FROM NumberOfSideEffectsByDrugCombination AS SideComb, NumberOfUsesByDrugCombination as NUses
WHERE SideComb.druga = NUses.druga and SideComb.drugb = NUSes.drugb


SELECT Comb.druga, Comb.drugb, Comb.sideeffectfreq/Alone.sideeffectfreq
FROM CombinedSideEffectFrequency as Comb, AloneSideEffectFrequency as Alone
WHERE Comb.druga = Alone.codeDrug;